In [ ]:
# Cell 1 — install (run in a notebook cell with ! prefix)
!pip install --upgrade pip
!pip install tensorflow matplotlib pillow pandas numpy 

In [ ]:
pip install tensorflow


In [ ]:
# Cell 2 - imports and model map
import os
import time
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.applications import (
    efficientnet,
    resnet50,
    densenet,
    vgg16
)
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input

# Map for models: (loader_fn, preprocess_fn, input_size)
MODEL_MAP = {
    "EfficientNetB0": (efficientnet.EfficientNetB0, efficientnet.preprocess_input, (224,224)),
    "ResNet50": (resnet50.ResNet50, resnet50.preprocess_input, (224,224)),
    "DenseNet121": (densenet.DenseNet121, densenet.preprocess_input, (224,224)),
    "VGG16": (vgg16.VGG16, vgg16.preprocess_input, (224,224)),
}

def load_model_by_name(name):
    loader, pfn, dims = MODEL_MAP[name]
    model = loader(weights='imagenet', include_top=True)
    return model, pfn, dims


In [ ]:
# Cell 3 - image helpers
def load_image(path, target_size):
    img = Image.open(path).convert('RGB')
    img = img.resize(target_size, Image.BILINEAR)
    arr = np.array(img).astype(np.float32)
    return img, arr

def show_image_from_array(arr, title=None):
    # arr: H,W,C in uint8 or float 0-255
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    plt.figure(figsize=(4,4))
    plt.imshow(arr)
    plt.axis('off')
    if title:
        plt.title(title)
    plt.show()


In [ ]:
# Cell 4 - preprocess & prediction helpers
def preprocess_for_model(raw_arr, preprocess_fn):
    # raw_arr: H,W,C in 0-255 floats
    x = np.expand_dims(raw_arr.copy(), axis=0)
    x = preprocess_fn(x)
    return x

def predict_and_decode(model, preprocess_fn, raw_arr, top=3):
    x = preprocess_for_model(raw_arr, preprocess_fn)
    preds = model.predict(x)
    decoded = decode_predictions(preds, top=top)[0]
    return preds, decoded


In [ ]:
# Cell 5 - attacks (FGSM & PGD)
@tf.function
def tf_fgsm(model, x, y_true, epsilon):
    # x: preprocessed tensor (1,H,W,C)
    # y_true: integer tensor shape (1,)
    with tf.GradientTape() as tape:
        tape.watch(x)
        logits = model(x, training=False)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, logits, from_logits=True)
    grad = tape.gradient(loss, x)
    signed_grad = tf.sign(grad)
    adv = x + epsilon * signed_grad
    adv = tf.clip_by_value(adv, -1.0, 1.0)  # works for [-1,1] preprocess
    return adv

def tf_pgd(model, x, y_true, epsilon, alpha, iters):
    x_orig = x
    adv = tf.identity(x)
    for i in range(iters):
        with tf.GradientTape() as tape:
            tape.watch(adv)
            logits = model(adv, training=False)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, logits, from_logits=True)
        grad = tape.gradient(loss, adv)
        adv = adv + alpha * tf.sign(grad)
        adv = tf.clip_by_value(adv, x_orig - epsilon, x_orig + epsilon)
        adv = tf.clip_by_value(adv, -1.0, 1.0)
    return adv


In [ ]:
# Cell 6 - display helpers & utilities
def inv_preprocess_for_display(adv_preprocessed, model_name):
    """
    Convert a preprocessed tensor back to image-like uint8 for display.
    This uses a simple normalization trick (works for many Keras preprocess_input functions),
    but it is approximate. For perfect inverse you'd need the exact preprocess transform.
    """
    arr = adv_preprocessed.copy()
    # arr is expected to be shape (1,H,W,C)
    a = np.squeeze(arr, axis=0)
    # If preprocess maps input to [-1,1], rescale:
    if (a.max() <= 1.1 and a.min() >= -1.1):
        disp = ((a + 1.0) * 127.5).astype(np.uint8)
    else:
        # fallback: min-max scale
        disp = (255.0 * (a - a.min()) / (a.max() - a.min() + 1e-8)).astype(np.uint8)
    return disp

def top1_info(preds):
    idx = np.argmax(preds[0])
    prob = tf.nn.softmax(preds, axis=-1).numpy()[0, idx]
    desc = decode_predictions(preds, top=1)[0][0][1]
    return desc, float(prob), int(idx)


In [ ]:
# Cell 7 - run single attack and epsilon sweep
import pandas as pd

def run_attack_single(model, preprocess_fn, raw_arr, attack='FGSM', epsilon=0.01, pgd_alpha=0.005, pgd_iters=10):
    x_pre = preprocess_for_model(raw_arr, preprocess_fn)
    preds_clean = model.predict(x_pre)
    clean_desc, clean_prob, clean_idx = top1_info(preds_clean)

    x_tf = tf.convert_to_tensor(x_pre, dtype=tf.float32)
    y_tf = tf.convert_to_tensor([clean_idx], dtype=tf.int64)

    if attack.upper() == 'FGSM':
        adv = tf_fgsm(model, x_tf, y_tf, float(epsilon))
    elif attack.upper() == 'PGD':
        adv = tf_pgd(model, x_tf, y_tf, float(epsilon), float(pgd_alpha), int(pgd_iters))
    else:
        raise ValueError("Unsupported attack (use FGSM or PGD)")

    preds_adv = model.predict(adv.numpy())
    adv_desc, adv_prob, adv_idx = top1_info(preds_adv)

    disp = inv_preprocess_for_display(adv.numpy(), None)

    result = {
        "attack": attack,
        "epsilon": float(epsilon),
        "clean_top1_desc": clean_desc,
        "clean_top1_prob": float(clean_prob),
        "adv_top1_desc": adv_desc,
        "adv_top1_prob": float(adv_prob),
        "timestamp": time.time()
    }
    return result, disp, preds_clean, preds_adv

def sweep_epsilon(model, preprocess_fn, raw_arr, attack='FGSM', eps_list=None, pgd_alpha=0.005, pgd_iters=10, show_plot=True):
    if eps_list is None:
        eps_list = np.linspace(0.0, 0.2, num=9)
    records = []
    x_pre = preprocess_for_model(raw_arr, preprocess_fn)
    preds_clean = model.predict(x_pre)
    clean_desc = decode_predictions(preds_clean, top=1)[0][0][1]
    for eps in tqdm(eps_list, desc="Sweeping ε"):
        res, _, _, preds_adv = run_attack_single(model, preprocess_fn, raw_arr, attack=attack, epsilon=eps, pgd_alpha=pgd_alpha, pgd_iters=pgd_iters)
        records.append({
            "epsilon": float(eps),
            "adv_top1_desc": res["adv_top1_desc"],
            "adv_top1_prob": res["adv_top1_prob"],
            "clean_top1_desc": res["clean_top1_desc"],
            "clean_top1_prob": res["clean_top1_prob"]
        })
    df = pd.DataFrame(records)
    if show_plot:
        plt.figure(figsize=(6,4))
        plt.plot(df["epsilon"], df["adv_top1_prob"], marker='o')
        plt.xlabel("Epsilon (ε)")
        plt.ylabel("Adversarial Top-1 Probability")
        plt.title(f"{attack} sweep")
        plt.grid(True)
        plt.show()
    return df

def save_results_csv(df, path):
    df.to_csv(path, index=False)
    print("Saved:", path)


In [ ]:
# Cell 8 - Example usage: change these paths/parameters for your machine

# 1) choose model name:
model_name = "EfficientNetB0"   # or "ResNet50", "DenseNet121", "VGG16"
model, preprocess_fn, target_size = load_model_by_name(model_name)
print("Loaded model:", model_name, "input size:", target_size)

# 2) provide an image path (replace with your local image)
image_path = "path/to/your/image.jpg"   # <-- change this
img, raw_arr = load_image(image_path, target_size)
print("Image loaded:", image_path)
show_image_from_array(raw_arr, title="Original image (resized)")

# 3) single FGSM attack
res, adv_disp, preds_clean, preds_adv = run_attack_single(
    model, preprocess_fn, raw_arr,
    attack='FGSM', epsilon=0.01
)
print("Single attack result:", res)
show_image_from_array(adv_disp, title=f"Adversarial (FGSM ε=0.01) - {res['adv_top1_desc']} {res['adv_top1_prob']:.3f}")

# 4) epsilon sweep example
eps_list = np.linspace(0.0, 0.05, num=6)   # small eps range to start
df = sweep_epsilon(model, preprocess_fn, raw_arr, attack='FGSM', eps_list=eps_list, show_plot=True)
print(df)

# 5) optional save
save_results_csv(df, "dl_robustness_results_sample.csv")
